# W266 Final Project

Authors: Satheesh Joseph, Catherine Mou, Yi Zhang

## Downloading and loading the data

We acquired the dataset from the researchers in the form of Sqlite `.db` files.

In [49]:
import os, sys, re, json, time, unittest
import itertools, collections
from importlib import reload

import numpy as np
from scipy import stats
import pandas as pd
import sqlite3

import nltk

In [50]:
# Download the files if they're not here
if 'data' not in os.listdir('.') or not os.listdir('data'):
    os.system('wget https://storage.googleapis.com/mids-w266-final-project-data/yelpHotelData.db -P data/')
    os.system('wget https://storage.googleapis.com/mids-w266-final-project-data/yelpResData.db -P data/')
    print('Data downloaded successfully!')
else:
    print('Already downloaded data')

Already downloaded data


In [51]:
con = sqlite3.connect('data/yelpResData.db')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

con = sqlite3.connect('data/yelpHotelData.db')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('review',), ('restaurant',), ('reviewer',)]
[('review',), ('sqlite_stat1',), ('sqlite_stat2',), ('reviewer',), ('hotel',)]


In [52]:
# Reading from the hotels database
hotels_db = sqlite3.connect("data/yelpHotelData.db")
hotels = pd.read_sql_query("SELECT * FROM hotel", hotels_db)
hotel_reviews = pd.read_sql_query("SELECT * FROM review", hotels_db)
hotel_reviewers = pd.read_sql_query("SELECT * FROM reviewer", hotels_db)


print(f'The data set contains {len(hotels)} hotels, {len(hotel_reviews)} reviews, and {len(hotel_reviewers)} reviewers')

The data set contains 283086 hotels, 688329 reviews, and 5123 reviewers


In [53]:
# Reading from the restaurant database
restaurant_db = sqlite3.connect("data/yelpResData.db")
restaurant_db.text_factory = lambda x: x.decode("utf-8", errors='ignore')
restaurants = pd.read_sql_query("SELECT * FROM restaurant", restaurant_db)
restaurant_reviews = pd.read_sql_query("SELECT * FROM review", restaurant_db)
restaurant_reviewers = pd.read_sql_query("SELECT * FROM reviewer", restaurant_db)


print(f'The data set contains {len(restaurants)} restaurants, {len(restaurant_reviews)} reviews, and {len(restaurant_reviewers)} reviewers')

The data set contains 242652 restaurants, 788471 reviews, and 16941 reviewers


# Exploratory Data Analysis

## ToDo: Performan EDA

In [54]:
hotel_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688329 entries, 0 to 688328
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   date           688329 non-null  object
 1   reviewID       688329 non-null  object
 2   reviewerID     688329 non-null  object
 3   reviewContent  688329 non-null  object
 4   rating         688329 non-null  int64 
 5   usefulCount    688329 non-null  int64 
 6   coolCount      688329 non-null  int64 
 7   funnyCount     688329 non-null  int64 
 8   flagged        688329 non-null  object
 9   hotelID        688329 non-null  object
dtypes: int64(4), object(6)
memory usage: 52.5+ MB


In [55]:
restaurant_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788471 entries, 0 to 788470
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   date           788471 non-null  object
 1   reviewID       788471 non-null  object
 2   reviewerID     788471 non-null  object
 3   reviewContent  788471 non-null  object
 4   rating         788471 non-null  int64 
 5   usefulCount    788471 non-null  int64 
 6   coolCount      788471 non-null  int64 
 7   funnyCount     788471 non-null  int64 
 8   flagged        788471 non-null  object
 9   restaurantID   788471 non-null  object
dtypes: int64(4), object(6)
memory usage: 60.2+ MB


In [56]:
reviews = pd.concat([restaurant_reviews, hotel_reviews.rename(columns={'hotelID':'restaurantID'})], ignore_index=True)
reviews.groupby('reviewerID').agg({"usefulCount": np.sum, 
                                   "coolCount": np.sum, 
                                   "funnyCount": np.sum}).sort_values(by=['usefulCount'], ascending=False)
reviews[reviews['reviewerID'] == 'w-w-k-QXosIKQ8HQVwU6IQ']['reviewContent']

94         ***Alinea is truly a one-of-a-kind experience;...
29403      ***Graham Elliot serves up refined casual food...
43054      ***Longman & Eagle is a true gastropub--a casu...
98176      ***Andrew Kirschner's Tar & Roses shows a lot ...
98177      ***Jose Andres' China Poblano in the Cosmopoli...
                                 ...                        
1214489    **Sleek, upscale and reminiscent of the flashy...
1214498    ***Great stadium design though the food was di...
1214499    (3.5 stars) Mastro's is well known for their s...
1214500    If you need something for your home improvemen...
1214501    I finally made it up to Alcove for breakfast a...
Name: reviewContent, Length: 3209, dtype: object

In [57]:
reviews.groupby('flagged').agg('sum')

,rating,usefulCount,coolCount,funnyCount
flagged,,,,
N,252531,65773,42690,36444
NR,3007835,1767423,1168514,995933
Y,34611,0,0,0
YR,2196506,868177,639502,516417
